<a href="https://colab.research.google.com/github/anhle/tensorFlow2.x/blob/master/Linear_Regression_bostonHousing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regressing 
the Boston housing dataset is real data and has 13 features. This is a regression problem because house prices—the label—we take as being continuously valued.
In a regression problem, we aim to predict the output of a continuous value, like a price or a probability. Contrast this with a classification problem, where we aim to select a class from a list of classes (for example, where a picture contains an apple or an orange, recognizing which fruit is in the picture).

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow==2.0.0-alpha0

import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
import  os

    100% |████████████████████████████████| 79.9MB 351kB/s 
    100% |████████████████████████████████| 61kB 22.7MB/s 
    100% |████████████████████████████████| 419kB 21.5MB/s 
    100% |████████████████████████████████| 3.0MB 11.6MB/s 


We scale the inputs to have mean 0 and standard variation 1.

In [0]:
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

In [0]:
def normalize(X_train, X_test):
    # this function normalize inputs for zero mean and unit variance
    # it is used when training a model.
    # Input: training set and test set
    # Output: normalized training set and test set according to the trianing set statistics.

    mean = np.mean(X_train)
    std = np.std(X_train)
    print('mean:', mean, 'std:', std)
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7)
    return X_train, X_test

In [4]:
def boston_dataset():

  (x_train, y_train), (x_val, y_val) = keras.datasets.boston_housing.load_data()

  x_train, x_val = x_train.astype(np.float32), x_val.astype(np.float32)
  print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
  x_train, x_val = normalize(x_train, x_val)
  
  db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64)
  db_val = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(102)
  
  return db_train,db_val

db_train,db_val = boston_dataset()

57344/57026 [==============================] - 0s 0us/step
(404, 13) (404,) (102, 13) (102,)
mean: 69.79277 std: 144.39195


Implement linear regression by sub-class keras.layers.Layer.

A linear regression is evaluated with an equation. The variable y is explained by one or many covariates. In your example, there is only one dependent variable. If you have to write this equation, it will be:

$$ y = wX + \beta$$


In [5]:
class Regressor(keras.layers.Layer):

    def __init__(self):
        super(Regressor, self).__init__()

        # here must specify shape instead of tensor !
        # name here is meanless !
        # [dim_in, dim_out]
        self.w = self.add_variable('meanless-name', [13, 1])
        # [dim_out]
        self.b = self.add_variable('meanless-name', [1])

        print(self.w.shape, self.b.shape)
        print(type(self.w), tf.is_tensor(self.w), self.w.name)
        print(type(self.b), tf.is_tensor(self.b), self.b.name)


    def call(self, x):

        x = tf.matmul(x, self.w) + self.b

        return x
      

model = Regressor()


(13, 1) (1,)
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> True meanless-name:0
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> True meanless-name:0


In [0]:
criteon = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=1e-2)

In [0]:
 for epoch in range(200):

      for step, (x, y) in enumerate(db_train):

          with tf.GradientTape() as tape:
              # [b, 1]
              logits = model(x)
              # [b]
              logits = tf.squeeze(logits, axis=1)
              # [b] vs [b]
              loss = criteon(y, logits)

          grads = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(grads, model.trainable_variables))

      print(epoch, 'loss:', loss.numpy())


      if epoch % 10 == 0:

          for x, y in db_val:
              # [b, 1]
              logits = model(x)
              # [b]
              logits = tf.squeeze(logits, axis=1)
              # [b] vs [b]
              loss = criteon(y, logits)

              print(epoch, 'val loss:', loss.numpy())

## Another way training data

### Define the loss and gradient function
Both training and evaluation stages need to calculate the model's loss. This measures how off a model's predictions are from the desired label, in other words, how bad the model is performing. We want to minimize, or optimize, this value.

* Our model will calculate its loss using the tf.keras.losses.SparseCategoricalCrossentropy function which takes the model's class probability predictions and the desired label, and returns the average loss across the examples.

* An optimizer applies the computed gradients to the model's variables to minimize the loss function. You can think of the loss function as a curved surface and we want to find its lowest point by walking around. The gradients point in the direction of steepest ascent—so we'll travel the opposite way and move down the hill. By iteratively calculating the loss and gradient for each batch, we'll adjust the model during training. Gradually, the model will find the best combination of weights and bias to minimize loss. And the lower the loss, the better the model's predictions.




Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

Train the model using tf.GradientTape:

##Training loop
With all the pieces in place, the model is ready for training! A training loop feeds the dataset examples into the model to help it make better predictions. The following code block sets up these training steps:

1. Iterate each epoch. An epoch is one pass through the dataset.
2. Within an epoch, iterate over each example in the training Dataset grabbing its features (x) and label (y).
3. Using the example's features, make a prediction and compare it with the label. Measure the inaccuracy of the prediction and use that to calculate the model's loss and gradients.
4. Use an optimizer to update the model's variables.
5. Keep track of some stats for visualization.
6. Repeat for each epoch.